## Desafio 04. Promoções cruzadas

### Bibliotecas

In [1]:
import pandas as pd

# Módulo de conexão com banco
import sys
sys.path.append('../') 
import funcoesconexao as con

# Desativa warnings
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados

#### Via banco de dados — SQL Server

In [2]:
conexao = con.conectar_bd()

----- Conexão com banco de dados estabelecida com sucesso! -----


In [3]:
# Carregando dataframes
df_pedidos  = con.carregar_dados(conexao, 'pedidos')
df_produtos = con.carregar_dados(conexao, 'produtos')

# Fechando conexão
con.fechar_conexao_bd(conexao)

In [4]:
# Exibindo dataframes
display(df_pedidos.head())
display(df_produtos.head())

,id,usuario_id,data_criacao,produto_id,quantidade
0,1,1,2019-01-01,1,10
1,2,3,2019-01-02,2,3
2,3,4,2019-01-02,2,25
3,4,5,2019-01-02,2,14
4,5,5,2019-01-06,3,14


,id,nome,preco
0,1,Pão de forma,10.15
1,2,Macarrão,4.20
2,3,Café,14.90
3,4,Vinho,89.90
4,5,Leite,6.10


#### Via arquivo .CSV

In [5]:
# Carregando dataframes
df_pedidos  = con.carregar_dados_csv('pedidos.csv')
df_produtos = con.carregar_dados_csv('produtos.csv')

In [6]:
# Exibindo dataframes
display(df_pedidos.head())
display(df_produtos.head())

,id,usuario_id,data_criacao,produto_id,quantidade
0,1,1,2019-01-01 00:00:00.000,1,10
1,2,3,2019-01-02 00:00:00.000,2,3
2,3,4,2019-01-02 00:00:00.000,2,25
3,4,5,2019-01-02 00:00:00.000,2,14
4,5,5,2019-01-06 00:00:00.000,3,14


,id,nome,preco
0,1,Pão de forma,"10,15"
1,2,Macarrão,"4,2"
2,3,Café,"14,9"
3,4,Vinho,"89,9"
4,5,Leite,"6,1"


### Solução

In [7]:
def promocoes_cruzadas(pedidos: pd.DataFrame, produtos: pd.DataFrame):    
    # Filtrando apenas pedidos a partir de 2022 
    pedidos_2022 = pedidos[pedidos['data_criacao'] >= '2022-01-01']
    
    # Mesclando dataframes
    pedidos_produtos = pd.merge(pedidos_2022, produtos, left_on='produto_id', right_on='id', how='inner')
    
    # Criando um novo dataframe para armazenar as combinações de produtos
    combinacoes = pd.merge(pedidos_produtos, pedidos_produtos, on='usuario_id', suffixes=('_01', '_02'))
    
    # Filtrando apenas as combinações por ordem alfabética
    combinacoes = combinacoes[combinacoes['nome_01'] < combinacoes['nome_02']]
    
    # Agrupando e contando as combinações de produtos
    combinacoes_contadas = combinacoes.groupby(['nome_01', 'nome_02']).size().reset_index(name='quantidade')
    
    # Ordenando as combinações por quantidade e nome do primeiro produto
    combinacoes_ordenadas = combinacoes_contadas.sort_values(by=['quantidade', 'nome_01'], ascending=[False, True])

    # Renomeando colunas
    combinacoes_ordenadas.rename(columns={'nome_01': 'produto_01', 'nome_02': 'produto_02'}, inplace=True)
    
    # Selecionando as cinco principais combinações
    top_5_combinacoes = combinacoes_ordenadas.head(5)
    
    return top_5_combinacoes

In [8]:
promocoes_cruzadas(df_pedidos, df_produtos)

,produto_01,produto_02,quantidade
3,Manteiga,Pão de forma,5
2,Macarrão,Molho de tomate,4
0,Café,Filtro descartável,3
4,Saca rolha,Vinho,2
1,Cereal matinal,Leite,1
